# Rover Basic Bench Test

In [ ]:
#!pip install pyserial

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ROVER_PORT = '/dev/ttyACM0'
ROVER_BAUDRATE = 115200

In [4]:
from rover import RoverClient
import time

In [8]:
rover = RoverClient(ROVER_PORT,ROVER_BAUDRATE)
rover.connect()

#### Test de conectividad

Encender/apagar led.

In [9]:
rover.led_on()
time.sleep(1)

In [10]:
rover.led_off()
time.sleep(1)

Header mismatch. Received packet: b'edOSadler \n'
Header mismatch. Received packet: b'\r ++\rHaFau   \n'
Header mismatch. Received packet: b'\r rdR  0:204 FFR F0  980R0 \r1:\n'
Header mismatch. Received packet: b'0:F Mrtaul\n'
Header mismatch. Received packet: b'E+3l \n'
Header mismatch. Received packet: b'4 F 010\r0: di\rtet12M\r:hI0cmm:sDO r\n'
Header mismatch. Received packet: b'Axa:0v//o3&-owot9 xt'
Header mismatch. Received packet: b'00:0:R  000  01F00  0   0A02odF ra01n0E8 1 84iriro)atoting =\xff'
Header mismatch. Received packet: b'\r\x00\r\r\r\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\n'
Header mismatch. Received packet: b'\x00\x00\x00\r\r\r\x00\r\x00\r\r\n'
Header mismatch. Received packet: b'\x00\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\x00\x00\r\r\x00\r\r\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\x00\x00\r\x00\r\x00\n'
Header mismatch. Received packet: b'\r\r\r\r\x00\r\x00\x00\r\r\x00\r\r\n'
H

Header mismatch. Received packet: b'\r\x00\r\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\x00\x00\r\r\r\x00\r\n'
Header mismatch. Received packet: b'\r\x00\r\x00\x00\n'
Header mismatch. Received packet: b'\r\x00\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\r\r\r\r\n'
Header mismatch. Received packet: b'\x00\x00\r\x00\r\r\r\x00\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\x00\x00\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\x00\r\r\r\r\r\n'
Header mismatch. Received packet: b'\x00\r\x00\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\n'
Header mismatch. Received packet: b'\x00\x00\r\r\r\r\n'
Header mismatch. Received packet: b'\x00\x00\x00\r\x00\r\r\r\r\x00\r\r\r\n'
Header mismatch. Received packet: b'\x00\x00\r\r\r\r\r\x00\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\r\x00\n'
Header mismatch. Received packet: b'\r\x00\r\x00\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\x00\r\r

Header mismatch. Received packet: b'\r\r\x00\r\r\r\r\x00\x00\r\r\n'
Header mismatch. Received packet: b'\r\x00\r\r\r\r\x00\x00\x00\x00\r\n'
Header mismatch. Received packet: b'\r\x00\x00\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\r\x00\r\n'
Header mismatch. Received packet: b'\r\r\x00\r\r\r\n'
Header mismatch. Received packet: b'\x00\r\x00\r\x00\r\r\n'
Header mismatch. Received packet: b'\r\x00\x00\r\r\n'
Header mismatch. Received packet: b'\x00\x00\r\r\r\x00\n'
Header mismatch. Received packet: b'\x00\r\r\r\x00\r\r\x00\n'
Header mismatch. Received packet: b'\r\r\r\r\r\n'
Header mismatch. Received packet: b'\x00\x00\x00\r\r\x00\r\n'
Header mismatch. Received packet: b'\x00\r\x00\r\r\r\r\x00\x00\x00\n'
Header mismatch. Received packet: b'\r\r\r\r\x00\r\r\n'
Header mismatch. Received packet: b'\r\x00\r\r\x00\r\n'
Header mismatch. Received packet: b'\x00\r\x00\r\r\x00\n'
Header mismatch. Received packet: b'\r\r\x00\x00\r\x00\r\x00\r\n'
Header mismatch. Received packet: b'\x00\r\r

Header mismatch. Received packet: b'\r\r\r\x00\r\n'
Header mismatch. Received packet: b'\x00\x00\r\x00\x00\r\r\x00\x00\r\x00\x00\r\r\r\r\x00\r\n'
Header mismatch. Received packet: b'\r\r\r\rf\r\n'
Header mismatch. Received packet: b'\r\r\r\xaf\r\n'
Header mismatch. Received packet: b'\x00\r\r\r\x00\r\x9f\n'
Header mismatch. Received packet: b'\r\r\r\r\r\xff\n'
Header mismatch. Received packet: b'\r\x00\x00\r\r\x00\x00\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\x00\r\x00\x00\n'
Header mismatch. Received packet: b'\r\r\x00\r\x00\x00\n'
Header mismatch. Received packet: b'\x00\r\r\r\x00\r\x00\x00\x00\n'
Header mismatch. Received packet: b'\r\r\x00\r\r\n'
Header mismatch. Received packet: b'\r\r\x00\r\r\n'
Header mismatch. Received packet: b'\r\x00\x00\r\r\r\n'
Header mismatch. Received packet: b'\r\r\x00\r\x00\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\x00\r\x00\r\r\r\n'
Hea

Header mismatch. Received packet: b'\xe7"\r\xff\r\r\r\r\xef\r\n'
Header mismatch. Received packet: b'\r\r\x9bb\r\n'
Header mismatch. Received packet: b'\x1d\r\r\r$\n'
Header mismatch. Received packet: Header mismatch. Received packet: b'\r\xd4\rx\r\xf8\r\r;u\r\x07\r\rI\x1e\r\r\r\r\r\xf9\r\r\n'
b'\x1f\r\r\r\r\r\n'
Header mismatch. Received packet: b'\xf0\xc0\r\x9c\r\n'
Header mismatch. Received packet: b"\r'\r\r\x84\r\r\n"
Header mismatch. Received packet: b'\rY\r\x97\r\n'
Header mismatch. Received packet: b'\r\r\xcb\r\r\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\xe0\rh\r\xfa\n'
Header mismatch. Received packet: b'\r\xd9\r\r\r\r\xfd\n'
Header mismatch. Received packet: b'\r\r\rm\x10\r\r\r\r\n'
Header mismatch. Received packet: b'\r\xa3\r\xa2\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\xe6t\n'
Header mismatch. Received packet: b'\r\r\r\xc4\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\n'
Header mismatch. Received packet: b'\r\xe1\x10b\r\n'
Header mismatch. Receiv

...

Header mismatch. Received packet: b'\r;\xec\r\r\x12\n'
Header mismatch. Received packet: b'\x90\r\r\r\r\n'
Header mismatch. Received packet: b'\x82\xda\r\r\r\r\n'
Header mismatch. Received packet: b'\xed\xd8\r\r\r\n'
Header mismatch. Received packet: b'\r\xf1a\xa2\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\xcf\r\r\r\r\n'
Header mismatch. Received packet: b'\x98\r\r\rs\r\rM\n'
Header mismatch. Received packet: b'\r8\r\x8a\r\r\r\r\n'
Header mismatch. Received packet: b'\rF\r\r\r\r\n'
Header mismatch. Received packet: b'\r\rOm\r\r\n'
Header mismatch. Received packet: b'\xf4\r\xe5\x9f\r\n'
Header mismatch. Received packet: b'\r\x10\r\r\xb6\r\r\xb1\xdc\r\n'
Header mismatch. Received packet: b'\r\r\r\r\xaaBI\r\r\n'
Header mismatch. Received packet: b'\r\r\r\xc7\r\n'
Header mismatch. Received packet: b'\r\x06\rI[\n'
Header mismatch. Received packet: b'\r\x19\r\r\r\r\x97\r\r\xf1\xc1\r\r\n'
Header mismatch. Received packet: b'\xaf\r\rO\r\r\n'
Header mismatch. Received packet: b'\xd7\

Header mismatch. Received packet: b'\r\r\xca\r\r\r\xb5\r\r\r\xec\r\rK\r\r\n'
Header mismatch. Received packet: b'\r\xc9\x03\rO\n'
Header mismatch. Received packet: b'\r\rI\r\r\r\xa4\r\n'
Header mismatch. Received packet: b'\r\xf2\r\r\r\r\n'
Header mismatch. Received packet: b'\r\xd0\r\xde\r\xce\rd\r\r\r\n'
Header mismatch. Received packet: b'\x03\xb4\xa3O\r\r\n'
Header mismatch. Received packet: b'\r\xdf\r\rH\r\r\xf5\\\r\r\n'
Header mismatch. Received packet: b'\x8b\xca\r\r\r\r\r\r\xa4-\n'
Header mismatch. Received packet: b'\r\r\r\r\x8e\r\r\r\r\r\r\r\r-\r\r\r\n'
Header mismatch. Received packet: b'h\r\x86\r\r\r\r\n'
Header mismatch. Received packet: b'\xdf\r\x1f\r\r\n'
Header mismatch. Received packet: b'\r\x11\xa6\r\r\n'
Header mismatch. Received packet: b'\x90s\r\rQ`\xe1\r\x99\r\x98\r\r\rn\rQ\xd0\x89\n'
Header mismatch. Received packet: b'\xe1\r\r\r\r\r\r\rx\xa6\n'
Header mismatch. Received packet: b'\xb6\r\r\r\r\xc6?\r\r\r\r\xd7\r\r\x9c\r\n'
Header mismatch. Received packet: b'\r\r

Header mismatch. Received packet: b'\r\x1e\r\rM\r\r\r\r\r\xcd\xc6\xac\n'
Header mismatch. Received packet: b'\r\r\r\r\xd3\r\n'
Header mismatch. Received packet: b'\x14\xd4\r\r)\x8f\r\n'
Header mismatch. Received packet: b'\r\r\r\x16\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\xa9\x92\r\n'
Header mismatch. Received packet: b'\r\r\x98g\xbf\n'
Header mismatch. Received packet: b'\r\xea\r\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\rq\xe5\xf9\n'
Header mismatch. Received packet: b'\r\xf2\x12\x82\r\r\r\n'
Header mismatch. Received packet: b'&\xa1\r\r\r\xc4\x15\r\r\r\r\xa4\r\r\r\n'
Header mismatch. Received packet: b'\x8c\x0b\x9e\xc6\r\n'
Header mismatch. Received packet: b'\xd0\r\r\r\xe3\n'
Header mismatch. Received packet: b'\r\r\r\rX\r\r\n'
Header mismatch. Received packet: b'\x83\r\r\xc7\xc1\r\r\n'
Header mismatch. Received packet: b'\r \r\r\x91\r\n'
Header mismatch. Received packet: b'\r\r\r8\r\n'
Header mismatch. Received packet: b'\r\xe7\xf1\r\xfb\n'
Header mismatc

Header mismatch. Received packet: b'\r\r\r\r\rX\x84\r\xb3\n'
Header mismatch. Received packet: b'A\r\r\r\r.\r\n'
Header mismatch. Received packet: b'\xe7b\r\r\r\r\r\r\r\n'
Header mismatch. Received packet: b'\xfd\r\r\r\r\n'
Header mismatch. Received packet: b'\r\r\x91\r\r\n'
Header mismatch. Received packet: b'\xe4\r\r\r\xbf\x80\r\n'
Header mismatch. Received packet: b'\r\xc9\r\r\r\r\r\rl\n'
Header mismatch. Received packet: b'\rW\r\x04\r9\n'
Header mismatch. Received packet: b'\r\xf0\r\x9d\r\n'
Header mismatch. Received packet: b'{\r\r\x15\r\r\r\r\xab\r\r\r\n'
Header mismatch. Received packet: b'\rFDr\xd7\x04\r\r\n'
Header mismatch. Received packet: b'\r\r\r\r\r\r\r\rb\x9c\n'
Header mismatch. Received packet: b'\r\xd9\r\r\r\r\n'
Header mismatch. Received packet: b'\r\xac\r\xb1\r\r\r\n'
Header mismatch. Received packet: b'\r\x16\r\x88\r\r\n'
Header mismatch. Received packet: b'\r \xb1\xca\rn\xf1\n'
Header mismatch. Received packet: b'\r\r\r\r}\r\r\n'
Header mismatch. Received packet: b

Header mismatch. Received packet: b'\r9\r\r\r\rS'
Header mismatch. Received packet: b'P\x00\x00K!\n'
Header mismatch. Received packet: b'T!K\x06T!\x06\x80TPTPTP!'
Header mismatch. Received packet:Header mismatch. Received packet: b'\x00K\x06\x80\x06K\x06\x00'
 b'T!K!\x06\x00TKT!\x80TKT!\x06\x80'
Header mismatch. Received packet: b'KK!TPKPT\x80P!PP!PP'
Header mismatch. Received packet: b'T\x00\x80P\x06PT'
Header mismatch. Received packet: b'PPKT!\x06\x80PKT\x06P\x80P\x06K\x80T\x06\x80P\x06\x80PK\x80K'
Header mismatch. Received packet: b'\x80P\x80\x00\x00\x80\x00'
Header mismatch. Received packet: b'P\x00PP\x80\x00\x00\x00K'
Header mismatch. Received packet: b'\x00P\x80PPP'
Header mismatch. Received packet: b'T\x00\x00\x00\x00\x00\x00'
Header mismatch. Received packet: b'PKTPKPPK\x00T'
Header mismatch. Received packet: b'K\x00P\x00\x00\x00P\x00P\x00\xda'
Header mismatch. Received packet: b'\x00\x00\x00PKTPKT!'
Header mismatch. Received packet: b'\x00K\x00\x00P\x00\x00'
Header mismatch. R

In [ ]:
rover.request_tmy()
time.sleep(1)

Leer telemetría

#### Low level motor control (no trajectory/PID)

In [ ]:
MAX_SPEED = 255

Move forward increasing velocity gradually.

In [ ]:
for i in range(0,MAX_SPEED):
    rover.update_motor_speeds( [i, i], RoverClient.MOTOR_A | RoverClient.MOTOR_B )
    time.sleep(0.1)

Decrease velocity gradually.

In [ ]:
for i in range(0,MAX_SPEED):
    rover.update_motor_speeds( [MAX_SPEED-i, MAX_SPEED-i], RoverClient.MOTOR_A | RoverClient.MOTOR_B )
    time.sleep(0.1)

Move in reverse increasing velocity gradually.

In [ ]:
for i in range(0,MAX_SPEED):
    rover.update_motor_speeds( [-i, -i], RoverClient.MOTOR_A | RoverClient.MOTOR_B )
    time.sleep(0.1)

Decrease velocity gradually.

In [ ]:
for i in range(0,MAX_SPEED):
    rover.update_motor_speeds( [-MAX_SPEED+i, -MAX_SPEED+i], RoverClient.MOTOR_A | RoverClient.MOTOR_B )
    time.sleep(0.1)

#### Tear down

Disconnect from Rover.

In [ ]:
rover.disconnect()